File used for features extraction for given url in csv format

In [20]:
import features_extractors.content_features as ctnfe
import features_extractors.url_features as urlfe
import features_extractors.external_features as trdfe
import features_extractors.feature_extractor as ftext
import sys
import pandas as pd
import torch
import numpy as np
from torch.utils.data import TensorDataset,DataLoader
from sklearn.model_selection import train_test_split
import pickle

In [21]:
header = [
        "url", "length_url", "length_hostname", "ip", "nb_dots", "nb_hyphens", "nb_at", "nb_qm", "nb_and", "nb_or", "nb_eq", "nb_underscore",
        "nb_tilde", "nb_percent", "nb_slash", "nb_star", "nb_colon", "nb_comma", "nb_semicolumn", "nb_dollar", "nb_space", "nb_www", "nb_com", "nb_dslash",
        "http_in_path", "https_token", "ratio_digits_url", "ratio_digits_host", "punycode", "port", "tld_in_path", "tld_in_subdomain",
        "abnormal_subdomain", "nb_subdomains", "prefix_suffix", "shortening_service", "path_extension", "nb_redirection",
        "nb_external_redirection", "length_words_raw", "char_repeat", "shortest_words_raw", "shortest_word_host", "shortest_word_path",
        "longest_words_raw", "longest_word_host", "longest_word_path", "avg_words_raw", "avg_word_host", "avg_word_path", "phish_hints",
        "domain_in_brand", "brand_in_subdomain", "brand_in_path", "suspecious_tld", "statistical_report", "nb_hyperlinks", "ratio_intHyperlinks",
        "ratio_extHyperlinks", "ratio_nullHyperlinks", "nb_extCSS", "ratio_intRedirection", "ratio_extRedirection", "ratio_intErrors",
        "ratio_extErrors", "login_form", "external_favicon", "links_in_tags", "submit_email", "ratio_intMedia", "ratio_extMedia", "sfh", "iframe",
        "popup_window", "safe_anchor", "onmouseover", "right_clic", "empty_title", "domain_in_title", "domain_with_copyright", "whois_registered_domain",
        "domain_registration_length", "domain_age", "web_traffic", "dns_record", "google_index", "page_rank", "status"
]

In [22]:
# Setup input

url = input("Give url:")
# url = 'https://www.youtube.com/'
print("Your url is: ", url)

Your url is:  https://www.google.pl/search?q=zxc&iflsig=AK50M_UAAAAAZAddinZ9CMLh84V7fHYVvRmFcilfcMRO&ved=0ahUKEwjNqufjiMr9AhWDuosKHWnECYUQ4dUDCAc&uact=5&sclient=gws-wiz


In [23]:
status = 0
features = ftext.extract_features(url, status)
print(features)

c:\Users\creep\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'input.payapi.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


['https://www.google.pl/search?q=zxc&iflsig=AK50M_UAAAAAZAddinZ9CMLh84V7fHYVvRmFcilfcMRO&ved=0ahUKEwjNqufjiMr9AhWDuosKHWnECYUQ4dUDCAc&uact=5&sclient=gws-wiz', 154, 13, 0, 2, 1, 0, 1, 4, 0, 5, 1, 0, 0, 3, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0.06493506493506493, 0.0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 15, 16, 1, 3, 1, 40, 6, 40, 8.6, 4.5, 9.23076923076923, 0, 1, 0, 0, 0, 0, 7, 0.5714285714285714, 0.42857142857142855, 0.0, 0, 0.0, 0.0, 0.0, 0.0, 0, 0, 100.0, 0, 100.0, 0.0, 0, 0, 0, 0.0, 0, 0, 0, 0, 1, 1, -1, 7474, 0, 0, 1, -1, 0]


In [24]:
savedModelName = 'phishing_v1.pt'
model = torch.load(savedModelName)
model.eval()

Sequential(
  (0): Linear(in_features=86, out_features=300, bias=True)
  (1): ReLU()
  (2): Linear(in_features=300, out_features=300, bias=True)
  (3): ReLU()
  (4): Linear(in_features=300, out_features=1, bias=True)
  (5): Sigmoid()
)

In [25]:
df = pd.DataFrame(data=[features], columns=header).drop("url", axis="columns")
df.head(1)

,length_url,length_hostname,ip,nb_dots,nb_hyphens,nb_at,nb_qm,nb_and,nb_or,nb_eq,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
0,154,13,0,2,1,0,1,4,0,5,...,0,1,1,-1,7474,0,0,1,-1,0


In [26]:
features = df.iloc[:, :-1]
is_phishing = df.iloc[:,-1:]

# Normalize the data
scaler = pickle.load(open('scaler.pkl', 'rb'))
feats_scaled = scaler.transform(features.values)
features = pd.DataFrame(data=feats_scaled, columns=features.columns)
features.head(1)

,length_url,length_hostname,ip,nb_dots,nb_hyphens,nb_at,nb_qm,nb_and,nb_or,nb_eq,...,empty_title,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank
0,0.08717,0.042857,0.0,0.043478,0.023256,0.0,0.333333,0.210526,0.0,0.263158,...,0.0,0.0,1.0,1.0,0.0,0.580941,0.0,0.0,1.0,-0.1


In [27]:
test_x_t=torch.from_numpy(features.values).float()
test_y_t=torch.from_numpy(is_phishing['status'].values).float().unsqueeze(1)

testingSet=TensorDataset(test_x_t,test_y_t)
testLoader = DataLoader(testingSet, batch_size=1, shuffle=True)

In [28]:
with torch.no_grad():
    for data in testLoader:
        inputs, labels = data
        outputs = model(inputs)
        predicted = torch.round(outputs.data)
        print(predicted)

tensor([[1.]])


In [29]:
if predicted == 1:
    print('The link might be phishing!')
else:
    print('The link is not phishing!')

The link might be phishing!
